<a href="https://colab.research.google.com/github/YuriSN/Challenge-Data-Science/blob/main/Notebook/Data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark Configuration

In [1]:
# Installing the Dependencies
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [3]:
import findspark
findspark.init()

# SparkSession

In [15]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Getting Started with Spark") \
    .config('spark.ui.port', '4050')\
    .getOrCreate()

In [16]:
spark

In [17]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [22]:
get_ipython().system_raw('./ngrok authtoken ngrok config add-authtoken 2Hbu6QsAExBSLe6OQGTWvXJsPDH_Q5nv3qKQ9KedU2whznkW')
get_ipython().system_raw('./ngrok http 4050 &')

In [23]:
!curl -s http://localhost:4040/api/tunnels

<html>
<head>
<meta http-equiv="Content-Type" content="text/html;charset=ISO-8859-1"/>
<title>Error 404 Not Found</title>
</head>
<body><h2>HTTP ERROR 404 Not Found</h2>
<table>
<tr><th>URI:</th><td>/api/tunnels</td></tr>
<tr><th>STATUS:</th><td>404</td></tr>
<tr><th>MESSAGE:</th><td>Not Found</td></tr>
<tr><th>SERVLET:</th><td>org.glassfish.jersey.servlet.ServletContainer-5394c943</td></tr>
</table>
<hr/><a href="https://eclipse.org/jetty">Powered by Jetty:// 9.4.48.v20220622</a><hr/>

</body>
</html>


In [21]:
spark

# Loading the Data

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
path = '/content/drive/MyDrive/Colab Notebooks/Challenger Data Science/semana-1/dataset_bruto.json'
data = spark.read.json(path)

In [ ]:
data.limit(5).toPandas()

# Analyzing the Data

In [94]:
from pyspark.sql.types import DoubleType, StringType, IntegerType, FloatType
from pyspark.sql.functions import explode, explode_outer
from pyspark.sql import functions as f


In [72]:
#Number of rows and columns
f'Rows: {data.count()}  Columns: {len(data.columns)}'

'Rows: 89083  Columns: 3'

In [ ]:
data.limit(5).toPandas()

In [ ]:
data.printSchema()

In [ ]:
anuncio=data.select('anuncio.*')
anuncio.show()

In [ ]:
anuncio.limit(500).toPandas()

In [ ]:
anuncio.printSchema()

Treatment of data types

In [143]:
anuncio = anuncio.withColumn('andar', anuncio['andar'].cast(IntegerType()))

In [144]:
anuncio = anuncio.withColumn('area_total', anuncio['area_total'][0].cast(IntegerType()))

In [145]:
anuncio = anuncio.withColumn('area_util', anuncio['area_util'][0].cast(IntegerType()))

In [146]:
anuncio = anuncio.withColumn('banheiros', anuncio['banheiros'][0].cast(IntegerType()))

In [147]:
anuncio = anuncio.withColumn('quartos', anuncio['quartos'][0].cast(IntegerType()))

In [148]:
anuncio = anuncio.withColumn('suites', anuncio['suites'][0].cast(IntegerType()))

In [149]:
anuncio = anuncio.withColumn('vaga', anuncio['vaga'][0].cast(IntegerType()))

In [150]:
anuncio = anuncio.withColumn('caracteristicas', anuncio['caracteristicas'][0].cast(StringType()))

Address Column Handling

In [ ]:
endereco = anuncio.select('endereco.*')
endereco.show()

In [ ]:
endereco = endereco.select('bairro', 'zona')
endereco.show()

In [151]:
#information about **bairro** and **zona** of the city being extracted.
anuncio = anuncio.withColumn('bairro', anuncio['endereco']['bairro'])
anuncio = anuncio.withColumn('zona', anuncio['endereco']['zona'])

In [152]:
#Deleting column "endereco"
anuncio = anuncio.drop('endereco')

Value Column Handling

In [153]:
anuncio = anuncio.withColumn('condominio', anuncio['valores']['condominio'][0].cast(IntegerType()))

In [154]:
anuncio = anuncio.withColumn('iptu', anuncio['valores']['iptu'][0].cast(IntegerType()))

In [155]:
anuncio = anuncio.withColumn('tipo', anuncio['valores']['tipo'][0].cast(StringType()))

In [156]:
anuncio = anuncio.withColumn('valor', anuncio['valores']['valor'][0].cast(IntegerType()))

In [157]:
#Deleting column "valores"
anuncio = anuncio.drop('valores')

Filtering by "Vendas"

In [ ]:
anuncio.where("tipo== 'Venda'")
anuncio.show()

Filtering by `tipo_uso`, `tipo_unidade` e `tipo_anuncio` 

In [ ]:
anuncio.groupBy(['tipo_uso', 'tipo_unidade', 'tipo_anuncio']).count().orderBy('count', ascending= False).show()

In [169]:
anuncio.where('tipo_uso == "Residencial" and tipo_unidade == "Apartamento" and tipo_anuncio == "Usado"').show()

+-----+----------+---------+---------+------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+----------+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|   caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|      zona|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+----------+----------+----+-----+-----+
|    3|        43|       43|        1|          Academia|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|           Paciência|Zona Oeste|       245|null|Venda|15000|
|    2|        42|       42|        1|     Churrasqueira|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|           Paciência|Zona Oeste|         0|   0|Venda|15000|
|    1|        

In [170]:
anuncio.printSchema()

root
 |-- andar: integer (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: integer (nullable = true)
 |-- iptu: integer (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: integer (nullable = true)



Data in parquet/csv format

In [171]:
anuncio.write.csv(
    path='/content/drive/MyDrive/Colab Notebooks/Challenger Data Science/semana-1/csv',
    mode='overwrite',
    header = True
)

In [178]:
#Test by checking runtime database saved in csv
%%time
anuncio2 = spark.read.csv(
    path='/content/drive/MyDrive/Colab Notebooks/Challenger Data Science/semana-1/csv',
    inferSchema = True,
    header=True
)

CPU times: user 10.9 ms, sys: 2.17 ms, total: 13.1 ms
Wall time: 1.01 s


In [ ]:
anuncio2.printSchema()

In [177]:
anuncio.write.parquet(
    path='/content/drive/MyDrive/Colab Notebooks/Challenger Data Science/semana-1/parquet',
    mode='overwrite'
)

In [179]:
#Test by checking runtime database saved in parquet
%%time
anuncio3 = spark.read.parquet(
    '/content/drive/MyDrive/Colab Notebooks/Challenger Data Science/semana-1/parquet'
)

CPU times: user 3.15 ms, sys: 946 µs, total: 4.1 ms
Wall time: 235 ms
